In [ ]:
#
# Tests the ability to use springs layout selectively on subgraphs
# ... so, if you keep the placement of the nodes from an initial all-nodes layout
# ... then you can update the placement of a subset of the nodes
#
# However... because that's not the only transform done to make this all work
# (think about the treemap component placement), then there's always a scaling factor
# involved... and that breaks the ability to apply the spring layout to a subset of 
# nodes...
#

import polars   as pl
import networkx as nx
import random
import copy
import rtsvg
import linknode_graph_patterns
from rtsvg.polars_spring_layout import PolarsSpringLayout
rt   = rtsvg.RACETrack()
g    = linknode_graph_patterns.LinkNodeGraphPatterns().__pattern_mesh__()
pos  = PolarsSpringLayout(g).results()
_lu_ = {'fm':[], 'to':[]}
for n in g.nodes():
    for nbor in g.neighbors(n): _lu_['fm'].append(n), _lu_['to'].append(nbor)
df        = pl.DataFrame(_lu_)
_relates_ = [('fm','to')]
_to_fix_  = {'node_0_5', 'node_7_4', 'node_8_6', 'node_5_8'}
_colors_  = {n:'red' for n in _to_fix_}
new_pos   = copy.deepcopy(pos)
for n in _to_fix_: new_pos[n] = (new_pos[n][0] + 100*(random.random()-0.5), new_pos[n][1] + 100*(random.random()-0.5))
new_new_pos = copy.deepcopy(new_pos)
new_new_pos = PolarsSpringLayout(g, pos=new_new_pos, static_nodes=set(g.nodes()) - _to_fix_).results()
params      = {'w':384, 'h':384, 'node_color':_colors_}
rt.tile([rt.link(df, _relates_, pos,         **params),
         rt.link(df, _relates_, new_pos,     **params),
         rt.link(df, _relates_, new_new_pos, **params)], spacer=10)

In [ ]:
#
# So... maybe it's possible to rescale the graph based on the expected distances
# ... in this case (mesh graph), all edges should be 1.0 ... so... how to take
# ... a random set of positions & then rescale them?
#
_d_sum_, _d_samples_ = 0.0, 0
for n in g.nodes():
    for nbor in g.neighbors(n):
        _w_                  = g[n][nbor]['weight']
        _xy_n_, _xy_nbor_    = pos[n], pos[nbor]
        _d_                  = rt.segmentLength((_xy_n_, _xy_nbor_))
        _d_sum_, _d_samples_ = _d_sum_ + _d_, _d_samples_ + 1
print(_d_sum_, _d_samples_, _d_sum_ / _d_samples_) # 359.24853900508407 288 1.2473907604343197 -- example from the mesh graph